In [1]:
import os
import myServices as ms
import randForReg as rfr
import numpy as np
import pandas as pd
import sklearn
import joblib

/Users/abdielfer/DESS/Internship2022/RNCanWork/FloodProbabRNCanAbd/randForReg.py:198: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=f"config", config_name="config.yaml")


In [2]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Random Forest Section

In [ ]:
sklearn.metrics.get_scorer_names()

In [23]:
DS = pd.read_csv('./basin1CleanToTrain.csv')

In [31]:
# DS.isna().sum()
# DS.dropna(inplace = True)
# DS.drop(columns=['x_coord', 'y_coord'],inplace = True)

In [38]:
DS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2479538 entries, 5 to 2493974
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
dtypes: float64(6), int64(1)
memory usage: 151.3 MB


In [34]:
DS['percentage'].unique()

array([0, 1, 5])

In [35]:
DS['percentage'].value_counts()

0    2426656
5      43820
1       9062
Name: percentage, dtype: int64

In [ ]:
# DS['weights'] = DS['percentage'].map({0:3, 1: 60, 5: 37})
sample = DS.sample(n=10000) # or use : frac=0.05
print(sample.head)
sample['percentage'].unique()

In [ ]:
sample['percentage'] = sample['percentage']*10

In [ ]:
print(sample['percentage'].unique())

In [ ]:
sample['percentage'].value_counts()

In [ ]:
sample.drop(columns=['x_coord', 'y_coord'],inplace = True)
print(sample.head)

In [ ]:
sample.to_csv('./sample.csv')

In [3]:
#Data import and exploration
datasetPath = './sample.csv'
rfReg  = rfr.implementRandomForestRegressor(datasetPath,'percentage', 0.2)
x_train,x_validation,y_train, y_validation = rfReg.getSplitedDataset()

In [ ]:
with timeit():
    bestEstimator = rfReg.fitRFRegressorWeighted(False)

In [ ]:
# import executeRFReg as exe
with timeit():
    rfr.main()

In [3]:
estimator = joblib.load('./models/rwReg/rfwgs_2207200845.pkl')

In [4]:
print(estimator)

RandomForestRegressor(max_depth=12, max_features=0.30000000000000004,
                      n_estimators=65, random_state=50)


In [10]:
weights = rfr.createWeightVector(y_validation,0,0.1)
y_hate = estimator.predict(x_validation)
r2 = sklearn.metrics.r2_score(y_validation,y_hate,sample_weight = weights)
print(r2)

-33.022138505829034


In [21]:
y_validation =  (np.array(y_validation).astype('float')).ravel()
print(rfr.validateWithR2(estimator,x_validation,
                         y_validation, False, 0,0.3))

-200.0360085221386


In [ ]:
k = ms.importConfig()

In [6]:
print(k[0])

'./sample.csv'

